<a href="https://colab.research.google.com/github/25Iqbalhossain/-Comparative-Analysis-of-AI-and-Signal-Processing-Techniques-for-Power-System-Condition-Monitoring/blob/main/Comparative_Analysis__Signal_Processing_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Comparative Analysis of AI and Signal Processing Techniques for Power System Condition Monitoring



In [1]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import pywt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 I Use Reference Materials **Dataset ** which  is Given GitHub Repository
link: [Datasets ](https://github.com/KingArthur000/Electrical-Fault-detection-and-classification)


In [4]:
# Load dataset
file_path = "/content/drive/MyDrive/DataSet(Comparative Analysis of AI and Signal Processing Techniques for Power System Condition Monitoring)/classData.csv"  # Ensure the correct file path
df = pd.read_csv(file_path)

In [5]:
# Select numerical features (currents and voltages)
X = df[['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc']].values

# Assuming "G" is the main target variable
y = df['G'].values

In [6]:
# Step 1: Wavelet Transform Feature Extraction
def wavelet_transform_with_stats(X):
    extracted_features = []
    for signal in X:
        coeffs = pywt.wavedec(signal, 'db4', level=3)  # 3-level wavelet decomposition
        features = np.hstack([np.mean(c) for c in coeffs] + [np.std(c) for c in coeffs])
        extracted_features.append(features)
    return np.array(extracted_features)

X_wavelet = wavelet_transform_with_stats(X)

# Step 2: Apply PCA for Dimensionality Reduction
pca = PCA(n_components=5)  # Keeping 5 principal components
X_reduced = pca.fit_transform(X_wavelet)

# Step 3: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.11/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [7]:
# Step 4: Train and Compare Multiple Models
# ANN Model (MLP Classifier)
from sklearn.neural_network import MLPClassifier # Import the MLPClassifier

ann = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=500, random_state=42)
ann.fit(X_train, y_train)
y_pred_ann = ann.predict(X_test)
print("ANN Performance:")
print(classification_report(y_test, y_pred_ann))
print(f"AUC-ROC Score: {roc_auc_score(y_test, y_pred_ann):.2f}\n")

ANN Performance:
              precision    recall  f1-score   support

           0       0.98      0.79      0.87       882
           1       0.78      0.98      0.87       691

    accuracy                           0.87      1573
   macro avg       0.88      0.88      0.87      1573
weighted avg       0.89      0.87      0.87      1573

AUC-ROC Score: 0.88



In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Define and train an ANN model
ann_model = MLPClassifier(hidden_layer_sizes=(50, 50), activation='relu', solver='adam', max_iter=500, random_state=42)
ann_model.fit(X_train, y_train)

# Evaluate ANN
y_pred_ann = ann_model.predict(X_test)
ann_accuracy = accuracy_score(y_test, y_pred_ann)
print(f'ANN Accuracy: {ann_accuracy:.4f}')

ANN Accuracy: 0.8601


In [9]:
# SVM as a Baseline ML Model
# Import the necessary class
from sklearn.svm import SVC

# SVM as a Baseline ML Model
svm = SVC(kernel='rbf', probability=True, random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("SVM Performance:")
print(classification_report(y_test, y_pred_svm))
print(f"AUC-ROC Score: {roc_auc_score(y_test, y_pred_svm):.2f}\n")


SVM Performance:
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       882
           1       0.84      0.86      0.85       691

    accuracy                           0.87      1573
   macro avg       0.87      0.87      0.87      1573
weighted avg       0.87      0.87      0.87      1573

AUC-ROC Score: 0.87



In [10]:
class SimpleANFIS:
    def __init__(self, num_rules=5):
        self.num_rules = num_rules
        self.weights = None

    def fit(self, X, y):
        self.weights = np.random.rand(X.shape[1], self.num_rules)  # Initialize random weights
        self.bias = np.random.rand(self.num_rules)

    def predict(self, X):
        # Calculate the weighted sum for each rule
        weighted_sum = np.dot(X, self.weights)

        # Add bias to the weighted sum and sum across rules
        output = np.sum(weighted_sum + self.bias, axis=1)

        # Round the output to the nearest integer for classification
        return np.round(output).astype(int)

In [11]:

# if i use this code SimpleANFIS with a basic gradient descent learning method:
# but here epochs 100 we train our model 100 times that why it's decreased
#AUC-ROC Score: 0.50 comes which is not good becouse
#AUC-ROC Score = 0.50 → The model performs like a random guess (no predictive power).

class SimpleANFIS:
    def __init__(self, num_rules=5, learning_rate=0.01, epochs=100):
        self.num_rules = num_rules
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        self.weights = np.random.rand(X.shape[1], self.num_rules)  # Initialize random weights
        self.bias = np.random.rand(self.num_rules)

        for epoch in range(self.epochs):
            # Forward pass
            weighted_sum = np.dot(X, self.weights)
            output = np.sum(weighted_sum + self.bias, axis=1)
            predictions = np.round(output).astype(int)

            # Calculate error
            error = y - predictions

            # Backpropagation to update weights and biases
            # Note: This is a simplified gradient descent
            self.weights += self.learning_rate * np.dot(X.T, error[:, np.newaxis]) / X.shape[0]
            self.bias += self.learning_rate * np.sum(error) / X.shape[0]

    def predict(self, X):
        weighted_sum = np.dot(X, self.weights)
        output = np.sum(weighted_sum + self.bias, axis=1)
        return np.round(output).astype(int)


In [12]:
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Handle class imbalance with SMOTE
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Train ANFIS Model with more rules
anfis = SimpleANFIS(num_rules=20)  # Increased rules for better learning
anfis.fit(X_train_resampled, y_train_resampled)

# Evaluate Model Performance
y_pred = np.round(anfis.predict(X_test_scaled))  # Round predictions to valid class labels
y_pred = np.clip(y_pred, 0, 16)  # Ensure predictions stay within valid range

print("ANFIS Performance:")
print(classification_report(y_test, y_pred))
print(f"AUC-ROC Score: {roc_auc_score(y_test, y_pred):.2f}")

print("Unique predictions:", set(y_pred))



ANFIS Performance:
              precision    recall  f1-score   support

           0       0.86      0.76      0.81       882
           1       0.73      0.85      0.79       691

    accuracy                           0.80      1573
   macro avg       0.80      0.80      0.80      1573
weighted avg       0.81      0.80      0.80      1573

AUC-ROC Score: 0.80
Unique predictions: {0, 1}


In [13]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.combine import SMOTETomek
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from scipy.spatial.distance import cdist

class RBNN:
    def __init__(self, num_centers, feedback_weight=0.5):
        self.num_centers = num_centers  # Number of RBF centers
        self.feedback_weight = feedback_weight  # Strength of recurrent connection
        self.centers = None
        self.sigma = None
        self.weights = None
        self.hidden_state = None  # Stores recurrent hidden state

    def rbf_function(self, X):
        """ Compute RBF kernel values with feedback """
        dist = cdist(X, self.centers)  # Compute Euclidean distance
        rbf_out = np.exp(-dist**2 / (2 * self.sigma**2))

        # Reset hidden_state for each prediction call
        # This ensures that the shape of hidden_state matches the input data during prediction
        self.hidden_state = None

        if self.hidden_state is not None:
            rbf_out += self.feedback_weight * self.hidden_state  # Add recurrent component

        self.hidden_state = rbf_out  # Store for next iteration
        return rbf_out

    def fit(self, X_train, y_train):
        """ Train RBNN using KMeans for centers and Logistic Regression for weights """
        kmeans = KMeans(n_clusters=self.num_centers, random_state=42)
        self.centers = kmeans.fit(X_train).cluster_centers_
        self.sigma = np.mean([np.linalg.norm(c - self.centers, axis=1) for c in self.centers])

        X_rbf = self.rbf_function(X_train)
        self.weights = LogisticRegression(max_iter=1000).fit(X_rbf, y_train)  # Train classifier

    def predict(self, X_test):
        """ Predict using RBNN """
        X_rbf = self.rbf_function(X_test)
        return self.weights.predict(X_rbf)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Handle class imbalance with SMOTETomek
smote_tomek = SMOTETomek()
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train_scaled, y_train)

# Train RBNN Model
rbnn = RBNN(num_centers=20, feedback_weight=0.5)  # Adjust the number of RBF centers & feedback
rbnn.fit(X_train_resampled, y_train_resampled)

# Evaluate Model Performance
y_pred = rbnn.predict(X_test_scaled)

print("\nRBNN Performance:")
print(classification_report(y_test, y_pred))
print(f"AUC-ROC Score: {roc_auc_score(y_test, y_pred):.2f}")
print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test, y_pred):.2f}")
print("Unique predictions:", set(y_pred))



RBNN Performance:
              precision    recall  f1-score   support

           0       0.88      0.87      0.87       882
           1       0.83      0.85      0.84       691

    accuracy                           0.86      1573
   macro avg       0.86      0.86      0.86      1573
weighted avg       0.86      0.86      0.86      1573

AUC-ROC Score: 0.86
Balanced Accuracy Score: 0.86
Unique predictions: {0, 1}
